In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from dotenv import load_dotenv
import nest_asyncio
import os

os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [4]:
import art
from art.local import LocalBackend
from rollout import ModelConfig

backend = LocalBackend()
model = art.TrainableModel(
    name="001",
    project="sweagent",
    config=ModelConfig(
        max_input_tokens=32_768,
        system_prompt_suffix="\n/no_think",
        xml_function_calling=True,
    ),
    base_model="Qwen/Qwen3-32B",
)
await model.register(backend)

INFO 05-31 16:56:11 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-31 16:56:11 [__init__.py:239] Automatically detected platform cuda.


/home/sky/sky_workdir/src/art/__init__.py:11: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-31 16:56:18 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-31 16:56:18 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-32b-bnb-4bit with actual GPU utilization = 78.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 139.72 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill toke

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.32it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.31it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.37it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.29it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.27it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.29i

INFO 05-31 16:56:35 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-31 16:56:35 [model_runner.py:1140] Model loading took 18.1555 GiB and 6.866419 seconds
INFO 05-31 16:56:41 [worker.py:287] Memory profiling takes 5.86 seconds
INFO 05-31 16:56:41 [worker.py:287] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.79) = 109.90GiB
INFO 05-31 16:56:41 [worker.py:287] model weights take 18.16GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 6.53GiB; the rest of the memory reserved for KV Cache is 85.07GiB.
INFO 05-31 16:56:41 [executor_base.py:112] # cuda blocks: 21777, # CPU blocks: 1536
INFO 05-31 16:56:41 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 10.63x
INFO 05-31 16:56:44 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the C

Capturing CUDA graph shapes: 100%|█████████████| 53/53 [00:49<00:00,  1.06it/s]


INFO 05-31 16:57:34 [model_runner.py:1592] Graph capturing finished in 50 secs, took 3.52 GiB
INFO 05-31 16:57:34 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 58.95 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


Unsloth 2025.5.1 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from instances import as_instances_iter, get_filtered_swe_smith_instances_df
from rollout import rollout

tasks = (
    get_filtered_swe_smith_instances_df()
    .sample(fraction=1.0, shuffle=True, seed=42)
    .pipe(as_instances_iter)
)

async for batch in art.trajectory_group_batches(
    (art.TrajectoryGroup(rollout(model, task) for _ in range(4)) for task in tasks),
    batch_size=4,
    # max_batch_exceptions=4,
    max_concurrent_batches=4,
    skip_batches=await model.get_step(),
):
    await model.train(batch)

batches:   0%|          | 0/16 [00:00<?, ?it/s]

CRITICAL:rex-deploy-asyncio_5:Traceback: 
Traceback (most recent call last):
  File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/swerex/runtime/local.py", line 310, in _run_normal
    expect_index = self.shell.expect(expect_strings, timeout=action.timeout)  # type: ignore
  File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/pexpect/spawnbase.py", line 354, in expect
    return self.expect_list(compiled_pattern_list,
  File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/pexpect/spawnbase.py", line 383, in expect_list
    return exp.expect_loop(timeout)
  File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/pexpect/expect.py", line 181, in expect_loop
    return self.timeout(e)
  File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/pexpect/expect.py", line 144, in timeout
    raise exc
pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /usr/bin/env
args: [b'/usr/bi

swerex.exceptions CommandTimeoutError
